<div style="float:left">
            <h1 style="width:450px">CASA0006 Practical 3: Tree-based methods</h1>
</div>
<div style="float:right"><img width="100" src="https://github.com/jreades/i2p/raw/master/img/casa_logo.jpg" /></div>

# Welcome!

In this workshop, we will achieve the following objectives:
1. Review weekly quiz;
1. Build three tree-based models to predict the daily bicycle rental based on metrics of R², including CART, random forest, and XGBoost;
1. Compare the predictive performance and permutation feature importance of these three models.

Remember that *Practice makes perfect.*

In the following code, some codes are missing and marked with ```??```.

Please replace ```??``` with the correct codes, using the hints and context. The solution will be given in the workshop.

\*\* Review the weekly quiz \*\*.

First, import the relevant libraries. 

* `pandas` for data import and handling;
* `matplotlib`;
* `numpy`;
* `sklearn`;
* `xgboost` for XGBoost models.
* `rfpimp` for permutation feature importance.

**Run the script below to get started.**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, validation_curve
from sklearn.metrics import root_mean_squared_error

# CART
from sklearn.tree import DecisionTreeRegressor

# random forest
from sklearn.ensemble import RandomForestRegressor

# feature importance
import rfpimp

# xgboost
import xgboost
from xgboost import XGBRegressor

pd.set_option('display.max_rows', 300) # specifies number of rows to show
pd.options.display.float_format = '{:40,.4f}'.format # specifies default number format to 4 decimal places
plt.style.use('ggplot') # specifies that graphs should use ggplot styling
%matplotlib inline

In [ ]:
# check the library version before we start
print("xgboost version:{}".format(xgboost.__version__))
print("sklearn version:{}".format(sklearn.__version__))

In [ ]:
# for recording runtime
import time
import psutil
import platform
import subprocess

start_time = time.time()

## Data Loading and Exploration

The dataset you will use relates to daily counts of rented bicycles from the bicycle rental company Capital-Bikeshare in Washington D.C., along with weather and seasonal information. The goal here is to predict how many bikes will be rented depending on the weather and the day. The original data can be downloaded from the [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset).

The dataset used in this workshop has been slightly processed by Christoph Molnar using the processing R-script from this [Github repository](https://github.com/christophM/interpretable-ml-book/blob/master/R/get-bike-sharing-dataset.R). Here, the dataset is provided as a csv file on Moodle.

Here is a list of the variables in the dataset:

- Count of bicycles including both casual and registered users. The count is used as the response in the regression task.
- Indicator of the season, either spring, summer, fall or winter.
- Indicator whether the day was a holiday or not.
- The year: either 2011 or 2012.
- Number of days since the 01.01.2011 (the first day in the dataset). This predictor was introduced to take account of the trend over time.
- Indicator whether the day was a working day or weekend.
- The weather situation on that day. One of:
  - **'GOOD'**: including clear, few clouds, partly cloudy, cloudy
  - **'MISTY'**: including mist + clouds, mist + broken clouds, mist + few clouds, mist
  - **'RAIN/SNOW/STORM'**: including light snow, light rain + thunderstorm + scattered clouds, light rain + scattered clouds, heavy rain + ice pallets + thunderstorm + mist, snow + mist
- Temperature in degrees Celsius.
- Relative humidity in percent (0 to 100).
- Wind speed in km/h.

We will use Pandas package to load and explore this dataset:

- Import the Boston housing dataset as a Pandas dataframe (call it `bike_rental`)
- Inspect the data
- Calculate summary statistics on all attributes

In [ ]:
bike_rental = pd.read_csv('https://raw.githubusercontent.com/huanfachen/Spatial_Data_Science/main/Dataset/daily_count_bike_rental.csv')
# drop the year variable as it is not useful
bike_rental = bike_rental.drop(['yr'], axis=1)

In [ ]:
bike_rental.info()

In `bike_rental`, there are two data types: categorical (aka `object`), and numerical (including `int64` and `float64`). 

In [ ]:
# print a few rows of this dataset
bike_rental.head()

### Converting categorical variables

Before undertaking regression, we need to handle categorical variables, as ML models usually can't deal with *strings* or *categories*.

While there are other ways of handling categorical variables in machine learning, we will stick to one-hot encoding, as it is easy to understand and appropriate for medium-size datasets (sample size <10000).

First, we need to convert categorical variables into dummy/indicator variables, using `One-Hot Encoding`.

In [ ]:
bike_rental_numeric = pd.get_dummies(bike_rental)

In [ ]:
# check out the new dataFrame
bike_rental_numeric.info()

Remember that, a cateogircal variable of K categories or levels, usually enters a regression as a sequence of K-1 dummy variables. The level that is left out becomes the reference level, and this is important for interpreting the regression model.

Here we manually choose the reference level for each categorical variable and exclude them from the DataFrame. You can change the reference levels if you want.

In [ ]:
bike_rental_final = bike_rental_numeric.drop(['season_SPRING', 'mnth_JAN', 'holiday_NO HOLIDAY', 'weekday_MON', 'workingday_WORKING DAY', 'weathersit_GOOD'], axis=1)

# double check the result
bike_rental_final.info()

### Splitting data into random train and test subsets

By default, `train_test_split` will split the data according to a 75:25 split. Other proportions can be specified, check out the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) for details.

Remember that the split should be random in order to avoid selection bias. Here, we set random_state=100 to guarantee reproducibility.

From the documentation:

The first argument of this function:

```
*arrays: sequence of indexables with same length / shape[0]
Allowed inputs are lists, numpy arrays, scipy-sparse matrices or pandas dataframes.
```

The output of this function:
```
splitting: list, length=2 * len(arrays)
List containing train-test split of inputs.
```

Here we input two dataframes (X and Y) and will get four outputs (train_x, test_x, train_y, test_y).

In [ ]:
random_state_split = 100
train_x, test_x, train_y, test_y = train_test_split(bike_rental_final.drop(['cnt'], axis = 1), bike_rental_final.cnt, random_state=random_state_split)

Double check the rows and columns of the outputs.

In [ ]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

# check the index of train_x and train_y - they should be identical. The index indicates which rows from the original data.

print(train_x.index.identical(train_y.index))
print(test_x.index.identical(test_y.index))

In the regression models, we will use the training set to train the model and select hyperparameters. The testing set is only used to report the performance of the finalised model.

## Building a CART

We will build a CART to predict the bicycle rental, following two steps:

1. Tune the hyperparameters of max_depth and min_samples_split using cross validation;
2. Train a final model based on the tuned hyperparameters and evaluate the model performance.

Here, we focus on two hyperparameters that control the stopping criteria of the model training: max_depth and min_samples_split.

*max_depth* refers to the maximal depth (or height) of a tree. In sklearn, if this value is not specified, the nodes would be expanded until all leaves contain samples of the same y value or until all leaves contain less than min_samples_split samples.

Intuition about max_depth: the larger the maximum tree depth, the better fitness of the model on the training data and thus the higher possibility of overfitting.

Ideally, we want the maximum tree height to be neither too small or too large.

The documentation is as follows:

```
max_depth: int, default=None
The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.
```

*min_samples_split* refers to the minimum number of samples required to split a node. If a node contains less samples than min_samples_split, this node becomes a leaf node and would not split.

The smaller value of min_samples_split, the better fitness of the model and thus the higher possibility of overfitting.

The documentation is as follows (note that the value can be either integer or float, but the meaning is different)

```
min_samples_split: int or float, default=2
The minimum number of samples required to split an internal node:

If int, then consider min_samples_split as the minimum number.

If float, then min_samples_split is a fraction and ceil(min_samples_split * n_samples) are the minimum number of samples for each split.
```

### Tuning the hyperparameters using cross validation

We will use ```GridSearchCV``` to find the optimal combination of max_depth and min_samples_split. 

Note that in grid search, all combinations of hyperparameters will be tested and compared. In this case, 25 (5*5) different combinations of hyperparameters will be tested.

**One question - how to determine the potential values of max_depth and min_samples_split?**

For min_samples_split, as it is a positive integer value and the default value is 2, we can set the range as [2,4,6,8,10].

For max_depth, we can build a decision tree using default settings and check the tree depth, then choose the range around this tree depth.

The code below shows that the tree depth of a CART with default settings is 25, so we can set the range as [10,20,30,40,50].

In [ ]:
# a CART using default settings
cart_default = DecisionTreeRegressor(random_state=0)
cart_default.fit(train_x, train_y)
# print the tree depth
print("Tree depth: {}".format(cart_default.get_depth()))

In [ ]:
# values of max_depth and min_samples_split
hyperparameters = {'max_depth':[10,20,30,40,50], 'min_samples_split':[2,4,6,8,10]}

randomState_dt = 10000
dt = DecisionTreeRegressor(random_state=randomState_dt)

# cv=5 by default, which means 5-fold cross-validation
clf = GridSearchCV(dt, hyperparameters)

clf.fit(train_x, train_y)

# we can query the best parameter value and its accuracy score
print ("The best parameter value is: ")
print (clf.best_params_)
print ("The best score is: ")
print (clf.best_score_)

### Training the final CART

In [ ]:
dt_final = DecisionTreeRegressor(max_depth=clf.best_params_['max_depth'], min_samples_split=clf.best_params_['min_samples_split'], random_state=randomState_dt)
dt_final.fit(train_x, train_y)

In [ ]:
print("R² on the training data:")
print(dt_final.score(X=train_x, y=train_y))
print("R² on the testing data:")
print(dt_final.score(X=test_x, y=test_y))

It looks like the R² on the testing data is much lower than that on the training data. This indicates the overfitting problem, meaning that the model fits very well to the training data but doesn't generalise well to unseen data.

In [ ]:
print("RMSE on the training data:")
print(root_mean_squared_error(train_y, dt_final.predict(train_x)))
print("RMSE on the testing data:")
print(root_mean_squared_error(test_y, dt_final.predict(test_x)))

The RMSE is in the same unit of the y variable (bicycle rental). On the testing data, on average, the model's prediction deviates from the true bicycle rental by 922.

If you want to know the relative error, you can compute the normalised RMSE by dividing the mean true value, as follows:

In [ ]:
print("RMSE on the training data:")
print(root_mean_squared_error(train_y, dt_final.predict(train_x))/np.mean(train_y))
print("RMSE on the testing data:")
print(root_mean_squared_error(test_y, dt_final.predict(test_x))/np.mean(test_y))

## Interpreting the CART

In [ ]:
# some attributes of the tree
print("Tree depth:{}".format(dt_final.??()))
print("Number of leaves:{}".format(dt_final.get_n_leaves()))

There are different ways to inspect and interpret the CART. 

One way is to visualise the whole tree and check each node and split. However, this is not easy for a tree with 10 layers and over 100 leaves.

Another way is to check the relative importance of each feature, for example, using the permutation feature importance.

To do this, we will use the **rfpimp** package. 

**rfpimp** is short for *random forest permutation importance*. It is a library that provides feature importances, based upon the permutation importance strategy, for general scikit-learn models and implementations specifically for random forest out-of-bag scores. Built by Terence Parr and Kerem Turgutlu.


We will use the **rfpimp.importances** function to compute the feature importance. According to [its documentation](https://github.com/parrt/random-forest-importances/blob/master/src/rfpimp.py):

```
Compute permutation feature importances for scikit-learn models using a validation set.
Given a Classifier or Regressor in model and validation X and y data, return a data frame with columns.
Feature and Importance sorted in reverse order by importance.
The validation data is needed to compute model performance measures (accuracy or R²). The model is not retrained.
```

So the importance of a feature x is defined as the difference of the R² model score (computed by **dt_final.score()**) after shuffling x and without shuffling x.

The feature importance is mostly positive or a negative value very close to zero, as shuffling a feature is not expected to greatly improve the model performance.

The feature importance is usually used to compare the relative importance of features. It is not recommended to interpret the absolute value of this importance.

In [ ]:
# the importances function returns a dataframe of two columns, Feature and Importance, and the features are ranked from the most important to the least.
imp = rfpimp.importances(dt_final, test_x, test_y)
print(imp)

In [ ]:
## Here is a barplot of the feature importance
viz = rfpimp.plot_importances(imp)
viz.view()

The feature importance above shows that the most important features are **days_since_2011**, **temperature**, and **humidity**.

Later on, we will compare the feature importance of diffferent models - CART, RF, and GBDT.

## Building and interpreting a random forest

In this part, we will use the random forest method to predict the daily bicycle rental. We will follow a simple workflow as CART: tune the hyperparameters and then train the final model.

RF has two groups of hyperparameters:

1. The hyperparameters of decision trees, such as max_depth, min_samples_split;
2. The configurations of RF, such as the number of trees (called **n_estimators**). By default, **n_estimators** is set as 100. Some intuition is that the larger n_estimators, the more stable the model performance and the longer computing time. This is because the computing time is a linear function of the number of trees.

The default value of 100 trees is good for most applications, so we don't tune this hyperparameter here.

As with CART, we will focus on the tuning of max_depth and min_samples_split.



In [ ]:
# values of max_depth and min_samples_split
hyperparameters = {'max_depth':[10,20,30,40,50], '??':[2,4,6,8,10]}

randomState_dt = 10000
rf = RandomForestRegressor(random_state=randomState_dt)

# cv=5 by default, which means 5-fold cross-validation
clf = GridSearchCV(rf, ??)

clf.fit(train_x, train_y)

# we can query the best parameter value and its accuracy score
print ("The best parameter value is: ")
print (clf.best_params_)
print ("The best score is: ")
print (clf.best_score_)

Train the final RF.

In [ ]:
rf_final = RandomForestRegressor(max_depth=clf.best_params_['max_depth'], min_samples_split=clf.best_params_['min_samples_split'], random_state=randomState_dt)
rf_final.fit(train_x, train_y)

In [ ]:
print("R² on the training data:")
print(rf_final.score(X=train_x, y=train_y))
print("R² on the testing data:")
print(rf_final.score(X=test_x, y=test_y))

So - what do you think about the comparison of R² on the training and testing data?

The R² on the testing data of RF is much higher than CART, meaning that RF is much better at making prediction for the unseen data and has a better generality. 

In [ ]:
print("RMSE on the training data:")
print(root_mean_squared_error(train_y, rf_final.??(??)))
print("RMSE on the testing data:")
print(root_mean_squared_error(test_y, rf_final.??(??)))

As with CART, it is not useful to visualise the trees in a random forest.

Calculate and plot the feature importance of the RF model.

Although the feature importance value differs from the CART, the order of the top-ranking variables is the same.

This indicates that these variables are indeed very important for the prediction of bike rental.

In [ ]:
imp = rfpimp.importances(rf_final, test_x, test_y)
print(imp)
viz = rfpimp.plot_importances(imp)
viz.view()

## Building and interpreting an XGBoost

In this part, we will use the XGBoost method to predict the daily bicycle rental. We will follow a simple workflow as CART and RF: tune the hyperparameters and then train the final model.

XGBoost is not part of the sklearn library but a separate library. It provides the sklearn API so that we can use XGboost in the same way as RandomForestRegressor of sklearn.

XGBoost has two groups of hyperparameters:

1. The hyperparameters of decision trees, such as max_depth, min_samples_split;
2. The configurations of XGBoost, such as the number of trees (called **n_estimators**) and regularisation hyperparameters.

Here are a list of potentially important hyperparameters for XGBoost, recommended by [this post](https://towardsdatascience.com/xgboost-fine-tune-and-optimize-your-model-23d996fab663):

1. `max_depth`: The maximum depth per tree. A deeper tree might increase the performance, but also the complexity and chances to overfit. The value must be an integer greater than 0. Default is 6.

1. `learning_rate`: The learning rate determines the step size at each iteration while your model optimizes toward its objective. A low learning rate makes computation slower, and requires more rounds to achieve the same reduction in residual error as a model with a high learning rate. But it optimizes the chances to reach the best optimum. The value must be between 0 and 1. Default is 0.3.

1. `n_estimators`: The number of trees in the ensemble. Equivalent to the number of boosting rounds. The value must be an integer greater than 0. Default is 100.

1. `colsample_bytree`: Represents the fraction of columns to be randomly sampled for each tree. It might improve overfitting.
The value must be between 0 and 1. Default is 1.

1. `subsample`: Represents the fraction of observations to be sampled for each tree. A lower values prevent overfitting but might lead to under-fitting.
The value must be between 0 and 1. Default is 1.

1. `alpha (reg_alpha)`: L1 regularization on the weights (Lasso Regression). When working with a large number of features, it might improve speed performances. It can be any integer. Default is 0.

1. `lambda (reg_lambda)`: L2 regularization on the weights (Ridge Regression). It might help to reduce overfitting. It can be any integer. Default is 1.

1. `gamma`: Gamma is a pseudo-regularisation parameter (Lagrangian multiplier), and depends on the other parameters. The higher Gamma is, the higher the regularization. It can be any integer. Default is 0.

The last three hyperparameters - alpha, lambda, and gamma - are related to the regularization of the model. In machine learning, regularization is a number of techniques to reduce the model complexity and the chances of overfitting.

If you want to know more about these hyperparameters, you can read [the XGBoost documentation](https://xgboost.readthedocs.io/en/latest/parameter.html).

Here, we will focus on the tuning of max_depth and n_estimators.

One note is that this grid search may take several minutes, which is longer than CART or random forest.

In [ ]:
# values of max_depth and min_samples_split
hyperparameters = {'??':[10,20,30,40,50], 'n_estimators':[50,100,150,200,250]}

randomState_xgb = 125
xgb = XGBRegressor(random_state=randomState_xgb)

# cv=5 by default, which means 5-fold cross-validation
gscv_xgb = GridSearchCV(xgb, hyperparameters)

gscv_xgb.fit(train_x, train_y)

# we can query the best parameter value and its accuracy score
print ("The best parameter value is: ")
print (gscv_xgb.best_params_)
print ("The best score is: ")
print (gscv_xgb.??)

Train the final XGBoost.

In [ ]:
xgb_final = XGBRegressor(max_depth=gscv_xgb.best_params_['max_depth'], n_estimators=gscv_xgb.best_params_['n_estimators'], random_state=randomState_xgb)
xgb_final.fit(train_x, train_y)

In [ ]:
print("R² on the training data:")
print(??.score(X=train_x, y=train_y))
print("R² on the testing data:")
print(xgb_final.score(X=??, y=test_y))

So - what do you think about the comparison of R² on the training and testing data?

XGBoost achieved a significantly higher R² score than CART and matched random forest on the same testing set, demonstrating stronger generalisation capabilities through its ensemble architecture that reduce overfitting to training data.

In [ ]:
print("RMSE on the training data:")
print(root_mean_squared_error(train_y, xgb_final.predict(train_x)))
print("RMSE on the testing data:")
print(root_mean_squared_error(test_y, xgb_final.??(??)))

As with CART, it seems not very useful to visualise the trees in a random forest.

In the following, we will calculate and plot the feature importance of the XGboost model.

Although the feature importance values differ from the CART, the order of the top-ranking variables is very similar with CART. This indicates that these top-ranking variables are indeed very important for the prediction of bike rental.

In [ ]:
imp = rfpimp.importances(xgb_final, ??, ??) # permutation
print(imp)
viz = rfpimp.plot_importances(imp)
viz.view()

## Comparing three models

we will collate and compare the model performance of the three models.

In [ ]:
# create a list of models
list_name_models = ['CART', 'RF', 'XGBoost']
# use the models from above
list_reg_models = [dt_final, rf_final, xgb_final]

dict_models = dict()

for name, model in zip(list_name_models, list_reg_models):
    dict_models[name] = [model.score(train_x, train_y), model.score(test_x, test_y), model.score(train_x, train_y) - model.score(test_x, test_y)]

# transform dict_models to dataframe
df_models = pd.DataFrame.from_dict(dict_models, orient='index', columns=['R²_train_data', 'R²_test_data', 'R²_diff'])
df_models
# you can then export df_models as a csv file and use it in MS Excel or Word
# df_models.to_csv(...)

The above result shows that the random forest model has a higher R² on the testing data than CART and XGBoost, as well as a smaller R² difference between training and testing data. 

This indicates that the random forest model is less subject to the overfitting issue and has a better generalisation.

The XGBoost also has a less extent of overfitting than CART. 

For this reason, it is uncommon to use CART to directly make predictions. Rather, CART is used to construct RF or GBDT.

In [ ]:
# %% [Runtime Report Cell]


def get_cpu_info():
    try:
        # Attempt Linux/WSL CPU info retrieval
        output = subprocess.check_output(
            'grep -m1 "model name" /proc/cpuinfo', 
            shell=True,
            stderr=subprocess.DEVNULL
        ).decode().strip()
        
        if 'model name' in output:
            # Extract GHz from string like "Intel(...) @ 2.40GHz"
            cpu_name = output.split(':')[1].strip()
            if '@' in cpu_name:
                ghz_part = cpu_name.split('@')[-1].replace('GHz', '').strip()
                return f"{ghz_part}GHz CPU", cpu_name
            return cpu_name.split('(')[0].strip(), cpu_name
    except (subprocess.CalledProcessError, FileNotFoundError):
        pass

    # Fallback 1: psutil detection
    try:
        if hasattr(psutil, "cpu_freq") and (freq := psutil.cpu_freq()):
            return f"{freq.max/1000:.2f}GHz CPU", "psutil-detected"
    except Exception:
        pass

    # Fallback 2: Manual specification
    return "1.80GHz CPU", "manual-default"  # Replace with your actual CPU speed

# Get kernel runtime
process = psutil.Process()
runtime_minutes = (time.time() - start_time) / 60

# Detect system specs
cpu_spec, cpu_source = get_cpu_info()

try:
    total_memory = psutil.virtual_memory().total / (1024**3)
    memory_spec = f"{total_memory:.1f}GB RAM"
except Exception:
    memory_spec = "8GB RAM"

print(f"Total runtime: {runtime_minutes:.1f} minutes\n"
      f"Hardware: {memory_spec}, {cpu_spec} ({cpu_source})\n"
      f"OS: {platform.system()} {platform.release()}")


## Summary

In this workshop, we have used decision trees, random forest, and gradient boosting decision trees to make predictions for the daily bicycle rental.

We compare the model performance and the feature importance. Overall, the RF and GBDT models have better predictive performance and better generalisation than CART.

The trend of relative feature importance is consistent across the three models. The most important features are days since 2011, temperature, and humidity.

In the later weeks, we will demonstrate the use of these models for classification tasks and the scalability of these models for big data.

## References and recommendations:

1. Permutation importance: [this chapter](https://christophm.github.io/interpretable-ml-book/feature-importance.html) is recommended, which is part of the E-book 'Interpretable machine learning'.
1. [The documentation](https://xgboost.readthedocs.io/en/latest/parameter.html) of XGBoost hyperparameters (note that they use the term *parameters* in this webpage).
